<a href="https://colab.research.google.com/github/jordan-gould/CS167Fall22Code/blob/main/Notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook #2: K Nearest Neighbors and Normalization
**Name: Jordan Gould**

**Due: 9/27 before class**

## Load in the data:
You'll notice that there are a lot of columns here. We're going to work with a subset of these columns, not the entire dataset.

In [35]:
#selecting columns in dataframes
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
#import the data:
#make sure the path on the line below corresponds to the path where you put your dataset.
spotify_data = pd.read_csv('/content/drive/MyDrive/CS167Datasets/spotify_top_charts.csv')
spotify_data.head()

,uri,artist_names,track_name,peak_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,time_signature,duration_ms
0,spotify:track:02MWAaffLxlfxAUY7c5dvx,Glass Animals,Heat Waves,1.0,0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,80.870,4.0,238805.0
1,spotify:track:5PjdY0CKGZdEuoNab3yDmX,"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),1.0,0.591,0.764,1.0,-5.484,1.0,0.0483,0.0383,0.000000,0.1030,169.928,4.0,141806.0
2,spotify:track:3IAfUEeaXRX9s9UdKOJrFI,Anitta,Envolver,3.0,0.812,0.736,4.0,-5.421,NaN,0.0833,0.1520,0.002540,0.0914,91.993,4.0,193806.0
3,spotify:track:1HhNoOuqm1a5MXYEgAFl8o,"Imagine Dragons, JID, Arcane, League of Legends",Enemy (with JID) - from the series Arcane Leag...,3.0,0.728,0.783,11.0,-4.424,0.0,0.2660,0.2370,0.000000,0.4340,77.011,4.0,173381.0
4,spotify:track:4fouWK6XVHhzl78KzQ1UjL,GAYLE,abcdefu,1.0,0.695,0.540,4.0,-5.692,1.0,0.0493,0.2990,0.000000,0.3670,121.932,4.0,168602.0


## Exercise #1: 
**Work with the right subset:**

You're not going to work with the whole data set, just the 'artist_names', 'track_name', 'peak_rank', 'duration_ms' and 2-4 additional features of your choice. To start, make this subset of the original data. 

Display the first five rows of your new subset. Also display the number of rows and columns of your subset.

In [37]:
spotify_subset = spotify_data[['artist_names', 'track_name', 'peak_rank', 'duration_ms', 'tempo', 'energy']]
spotify_subset.head()

,artist_names,track_name,peak_rank,duration_ms,tempo,energy
0,Glass Animals,Heat Waves,1.0,238805.0,80.870,0.525
1,"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),1.0,141806.0,169.928,0.764
2,Anitta,Envolver,3.0,193806.0,91.993,0.736
3,"Imagine Dragons, JID, Arcane, League of Legends",Enemy (with JID) - from the series Arcane Leag...,3.0,173381.0,77.011,0.783
4,GAYLE,abcdefu,1.0,168602.0,121.932,0.540


## Exercise #2:
**Check for null values**: If you have a null target value ('peak_rank'), you will need to throw that example out. 

Replace each of the null values within your predictors with the average of the column.

*hint: Useful functions here are `isna()`, `any()`, `fillna()`, `notna()` and `dropna()`. If you're not sure what these do, look them up in the [pandas documentation](https://pandas.pydata.org/docs/).*

After you have eliminated null values, use `.isna().any()` to verify that your fields no longer contain any null values (all of the values should be False).

In [71]:
spotify_subset.isna()

peak_rank_mean = spotify_subset['peak_rank'].mean()

spotify_subset['peak_rank'].fillna(peak_rank_mean, inplace = True)

spotify_subset.isna().any()

artist_names    False
track_name      False
peak_rank       False
duration_ms      True
tempo            True
energy           True
dtype: bool

## Exercise #3
Write up a k-nearest-neighbors function like the one you made for the iris data set in class. It should be able to make peak rank predictions for new songs. You should also be able to specify what you want to use as k.

You will need to incorporate the predictor features that you have chosen to be a part of your subset in your knn function (but not the artist_names nor track_name).

Pay special attention that I want you to refer to the target variable 'peak_rank' as a value (not a category). Thus, you need to adapt the code to handle a regression.

In [39]:
import numpy

def knn(song, data, k):
  data_copy = data.copy()
  data_copy['distance_to_new'] = numpy.sqrt(
     (song['duration_ms']-data['duration_ms'])**2
    +(song['tempo']-data['tempo'])**2
    +(song['energy']-data['energy'])**2)
  
  sorted_data = data_copy.sort_values(['distance_to_new']) 
  return sorted_data.iloc[0:k]['peak_rank'].mean()

## Exercise #4
Demonstrate that your function works by using the values from a song you recognize on the list. Pass in k=1 in to the funtion, and your algorithm should predict the exact peak_rank as your example song.

Make up some new values for a hypothetical new song.  Use your knn function to display a predicted peak_rank for the song. In this case, use a k value that is greater than 1.

In [73]:
# The song I chose was Everybody Wants to Rule the World by Tears for Fears
song = {}
song['duration_ms'] = 251489
song['tempo'] = 112.067
song['energy'] = .795

print("knn: ", knn(song, spotify_subset, 1))

knn:  105.0


The knn function correctly printed the exact peak_rank of the song I chose

## Exercise #5

Make a copy of the data and normalize the training data using Z-score.

*hint: `.copy()` will be useful here.* 

Print out the first 5 lines of the normalized subset.

Predict the peak rank with some input parameters using your k-nearest-neighbors function with both the normalized and non-normalized training data. Compare your results. Use a markup cell to describe and explain the cause of any differences. Detailed responses will receive full credit.

In [72]:
duration_mean = spotify_subset['duration_ms'].mean()
duration_std = spotify_subset['duration_ms'].std()

tempo_mean = spotify_subset['tempo'].mean()
tempo_std = spotify_subset['tempo'].std()

energy_mean = spotify_subset['energy'].mean()
energy_std = spotify_subset['energy'].std()

# normalized data set
norm_data = spotify_subset.copy()
norm_data['duration_ms'] = (norm_data['duration_ms'] - duration_mean) / duration_std
norm_data['tempo'] = (norm_data['tempo'] - tempo_mean) / tempo_std
norm_data['energy'] = (norm_data['energy'] - energy_mean) / energy_std

norm_data.iloc[0:5]

,artist_names,track_name,peak_rank,duration_ms,tempo,energy
0,Glass Animals,Heat Waves,1.0,0.639003,-1.423920,-0.697153
1,"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),1.0,-1.121217,1.722008,0.742479
2,Anitta,Envolver,3.0,-0.177584,-1.031005,0.573819
3,"Imagine Dragons, JID, Arcane, League of Legends",Enemy (with JID) - from the series Arcane Leag...,3.0,-0.548233,-1.560237,0.856926
4,GAYLE,abcdefu,1.0,-0.634956,0.026574,-0.606799


In [74]:
# I used the same song data as in exercise 4
norm_song = {}
norm_song['duration_ms'] = (251489 - duration_mean) / duration_std
norm_song['tempo'] = (112.067 - tempo_mean) / tempo_std
norm_song['energy'] = (.795 - energy_mean) / energy_std

print('non-normalized knn: ', knn(song, spotify_subset, 3))
print('normalized knn: ', knn(norm_song, norm_data, 3))

non-normalized knn:  74.66666666666667
normalized knn:  100.0


Since I used the same data as in example 4 for this problem, I actually know the peak_rank value this song has, which is 105.0. 

When comparing that to the non-normalized prediction for k = 3, its clear that the prediction is very different, since the value is 74.67. Comparing that to the normalized knn (which is 100) it is clear that the normalized value was much more accurate to the actual value. 

I feel like this level of difference is due to the fact that the non-normalized data has much more room for outliers to affect the results. Especially with songs that have a longer duration than average (which my song choice does). Once the data is normalized, there is much less room for the outliers to affect the results.  

## Exercise #6
For my cool thing, I wanted to look at all tracks on the list by The Weeknd and compare them to all tracks by Post Malone.

The value that I compared was the song energy, and I wanted to see the mean, standard deviation, min, and max for each artist. At the end I printed all the values so I could see how they related to each other.

In [75]:
weeknd_subset = spotify_subset[(spotify_subset['artist_names'] == 'The Weeknd')]
weeknd_energy_mean = weeknd_subset['energy'].mean()
weeknd_energy_max = weeknd_subset['energy'].max()
weeknd_energy_min = weeknd_subset['energy'].min()
weeknd_energy_std = weeknd_subset['energy'].std()

weeknd_subset['energy'].fillna(weeknd_energy_mean, inplace = True)

post_subset = spotify_subset[(spotify_subset['artist_names'] == "Post Malone")]
post_energy_mean = post_subset['energy'].mean()
post_energy_max = post_subset['energy'].max()
post_energy_min = post_subset['energy'].min()
post_energy_std = post_subset['energy'].std()

post_subset['energy'].fillna(post_energy_mean, inplace = True)

print('The average song energy by The Weeknd is: ', weeknd_energy_mean)
print('The maximum song energy by The Weeknd is: ', weeknd_energy_max)
print('The minimum song energy by The Weeknd is: ', weeknd_energy_min)
print('The standard deviation in song energy by The Weeknd is: ', weeknd_energy_std)

print('The average song energy by Post Malone is: ', post_energy_mean)
print('The maximum song energy by Post Malone is: ', post_energy_max)
print('The minimum song energy by Post Malone is: ', post_energy_min)
print('The standard deviation in song energy by Post Malone is: ', post_energy_std)


The average song energy by The Weeknd is:  0.6493
The maximum song energy by The Weeknd is:  0.959
The minimum song energy by The Weeknd is:  0.418
The standard deviation in song energy by The Weeknd is:  0.1446309641448666
The average song energy by Post Malone is:  0.5645000000000001
The maximum song energy by Post Malone is:  0.762
The minimum song energy by Post Malone is:  0.303
The standard deviation in song energy by Post Malone is:  0.16666943331037037


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
